##### Copyright 2025 The AI Edge Torch Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Exporting Gemma3 270M with AI Edge Torch

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-ai-edge/ai-edge-torch/blob/main/ai_edge_torch/generative/colabs/Gemma3_270M_convertion.ipynb)

In this colab, we will show you how to export a Gemma-3-270M model to LiteRT-LM format with AI Edge Torch.

It works with the base Gemma-3-270M-it model and its fine-tuned models. For later, checkout the [Full Model Fine-Tune using Hugging Face Transformers](https://ai.google.dev/gemma/docs/core/huggingface_text_full_finetune) tutorial.

#Prerequisite for exporting google/gemma-3-270m-it

- Create HuggingFace token with permission access to
  - google/gemma-3-270m-it

  This is needed to download the checkpoint and tokenizer.

- Open Colab Secrets: In your Google Colab notebook, locate the Secrets icon in the left-hand sidebar and click on it.
- Add a new secret: Click the "Add Secret" button.
- Name your secret: Enter "HF_TOKEN" for your token in the "Name" field.
- Paste your token: In the "Value" field, paste the actual token you want to store.

#Prerequisite for exporting fine-tuned model

- Access to the finetuned repo in Hugging Face Hub, or

- Access to the finetuned checkpoint


## Note: When running notebooks in this repository with Google Colab, some users may see the following warning message:

![Colab warning](https://github.com/google-ai-edge/ai-edge-torch/blob/main/docs/data/colab_warning.jpg?raw=true)

Please click `Restart Session` and run again.


This colab works with a free tier colab runtime.


In [ ]:
# 1. Uninstall existing tensorflow versions to prevent conflicts
!pip uninstall -y tensorflow tf-nightly

# 2. Install the requirements
!pip install -r https://raw.githubusercontent.com/google-ai-edge/ai-edge-torch/main/requirements.txt

# 3. Install ai-edge-torch (if you haven't already)
!pip install ai-edge-torch-nightly

Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 159.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 133.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 172.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.8 MB/s eta 0:00:00
   

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 523.6/523.6 kB 16.4 MB/s eta 0:00:00


In [ ]:
!pip install -U torchao

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 92.5 MB/s eta 0:00:00
  Attempting uninstall: torchao
    Found existing installation: torchao 0.10.0
    Uninstalling torchao-0.10.0:
      Successfully uninstalled torchao-0.10.0


In [ ]:
pip install --upgrade "jax[cuda]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 20.3 MB/s eta 0:00:00
  Attempting uninstall: jax-cuda12-pjrt
    Found existing installation: jax-cuda12-pjrt 0.5.3
    Uninstalling jax-cuda12-pjrt-0.5.3:
      Successfully uninstalled jax-cuda12-pjrt-0.5.3
  Attempting uninstall: nvidia-cuda-nvcc-cu12
    Found existing installation: nvidia-cuda-nvcc-cu12 12.5.82
    Uninstalling nvidia-cuda-nvcc-cu12-12.5.82:
      Successfully uninstalled nvidia-cuda-nvcc-cu12-12.5.82
  Attempting uninstall: jax-cuda12-plugin
    Found existing installation: jax-cuda12-plugin 0.5.3
    Uninstalling jax-cuda12-plugin-0.5.3:
      Successfully uninstalled jax-cuda12-plugin-0.5.3


In [ ]:
# Setup Hugging Face Hub credentials

import os
from google.colab import userdata
#.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
import os
os.environ['JAX_PLATFORMS'] = 'cuda'

import jax

In [ ]:
# @title Import needed packages.
from huggingface_hub import snapshot_download
from ai_edge_torch.generative.examples.gemma3 import gemma3
from ai_edge_torch.generative.utilities import converter
from ai_edge_torch.generative.utilities.export_config import ExportConfig
from ai_edge_torch.generative.layers import kv_cache

/usr/local/lib/python3.12/dist-packages/torch/distributed/distributed_c10d.py:351: UserWarning: Device capability of jax unspecified, assuming `cpu` and `cuda`. Please specify it via the `devices` argument of `register_backend`.
  warnings.warn(


# Exporting the checkpoint to LiteRT-LM format.

In this example, we directly use the google/gemma-3-270m-it repo. But you can also replace it with your fine-tuned model directory or repo ID.

If you are following the fine-tune colab and storing your checkpoint to Google Drive as the default setup, you can point to the checkpoint with the followings instead of downloading the base checkpoint.

```
from google.colab import drive
drive.mount('/content/drive')
checkpoint_dir = '/content/drive/MyDrive/MyGemmaNPC'
```


In [ ]:
# @title Download checkpoint

checkpoint_dir = snapshot_download('unsloth/gemma-3-270m-it')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

...

README.md: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

# Convert to LiteRT-LM format

After the following cell, you will be able to download the exported `.litertlm` file under `/contents/`, which will be accessible from the `Files` pannel.

In [ ]:
output_dir = '/content/'

# Import the weights and build the PyTorch model
pytorch_model = gemma3.build_model_270m(checkpoint_dir)

# Setup the export configurations and parameters for text generation models.
export_config = ExportConfig()
export_config.kvcache_layout = kv_cache.KV_LAYOUT_TRANSPOSED
export_config.mask_as_input = True

# Configs specific for text generation models.
litertlm_config = {
    "tokenizer_model_path": os.path.join(checkpoint_dir, 'tokenizer.model'),
    "start_token_id": 2,  # "<bos>"
    "stop_token_ids": [1, 106],  # ["<eos>", "<end_of_turn>"]
    "prompt_prefix": "<start_of_turn>user\n",
    "prompt_suffix": "<end_of_turn>\n<start_of_turn>model\n",
    "model_prompt_prefix": "<start_of_turn>model\n",
    "model_prompt_suffix": "<end_of_turn>\n",
    "user_prompt_prefix": "<start_of_turn>user\n",
    "user_prompt_suffix": "<end_of_turn>\n",
    "output_format": "litertlm",
}

# Convert to LiteRT or LiteRT-LM Format
converter.convert_to_litert(
    pytorch_model,
    output_path=output_dir,
    output_name_prefix="gemma",
    prefill_seq_len=2048,
    kv_cache_max_len=4096,
    quantize="dynamic_int8",
    export_config=export_config,
    **litertlm_config
)


In [ ]:
!pip freeze

absl-py==1.4.0
absolufy-imports==0.3.1
accelerate==1.10.1
ai-edge-litert-nightly==2.2.0.dev20260121
ai-edge-quantizer-nightly==0.5.0.dev20260122
ai-edge-torch-nightly==0.8.0.dev20260122
aiofiles==24.1.0
aiohappyeyeballs==2.6.1
aiohttp==3.13.0
aiosignal==1.4.0
alabaster==1.0.0
albucore==0.0.24
albumentations==2.0.8
ale-py==0.11.2
alembic==1.16.5
altair==5.5.0
annotated-types==0.7.0
antlr4-python3-runtime==4.9.3
anyio==4.11.0
anywidget==0.9.18
argon2-cffi==25.1.0
argon2-cffi-bindings==25.1.0
array_record==0.8.1
arrow==1.3.0
arviz==0.22.0
astropy==7.1.0
astropy-iers-data==0.2025.10.6.0.35.25
astunparse==1.6.3
atpublic==5.1
attrs==25.4.0
audioread==3.0.1
Authlib==1.6.5
autograd==1.8.0
babel==2.17.0
backcall==0.2.0
backports.strenum==1.2.8
beartype==0.22.2
beautifulsoup4==4.13.5
betterproto==2.0.0b6
bigframes==2.24.0
bigquery-magics==0.10.3
bleach==6.2.0
blinker==1.9.0
blis==1.3.0
blobfile==3.1.0
blosc2==3.10.0
bokeh==3.7.3
Bottleneck==1.4.2
bqplot==0.12.45
branca==0.8.2
Brotli==1.1.0
build

In [ ]:
!